# Download recordings for before PD diagnosis

In [ ]:
%cd /content/Project/



/content/Project


In [ ]:
ls

ParkCeleb_filtered/  SSL4PR/


In [ ]:
! git clone https://github.com/K-STMLab/SSL4PR.git

Cloning into 'SSL4PR'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 89 (delta 38), reused 81 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 86.66 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [ ]:
! pip install -r SSL4PR/requirements.txt

  Cloning https://github.com/sashank-tirumala/yaml_config_override.git to /tmp/pip-req-build-mzki_qk9
  Running command git clone --filter=blob:none --quiet https://github.com/sashank-tirumala/yaml_config_override.git /tmp/pip-req-build-mzki_qk9
  Resolved https://github.com/sashank-tirumala/yaml_config_override.git to commit 8b1cca49dd8a006a8c10a15cb54c831a33606f29
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.0/726.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 16.1 MB/s eta 0:00:00
  Created wheel for yaml_config_override: filename=yaml_config_override-0.6-py3-none-any.whl size=3329

In [ ]:
import pandas as pd
import os
import shutil
import re
import random
import json
from collections import defaultdict
from collections import Counter

In [ ]:
def filter_target_speakers(base_path, output_path):
    """
    Filters the PARKCELEB dataset to keep only target speakers with
    pre-diagnosis samples (before_after_diagnosis = "before" & years_from_diagnosis < 0).

    Parameters:
    - base_path: Path to the original PARKCELEB dataset.
    - output_path: Path to save the filtered dataset.
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for condition in ["PD"]:  # Parkinson groups
        condition_path = os.path.join(base_path, condition)
        output_condition_path = os.path.join(output_path, condition)

        for speaker_folder in os.listdir(condition_path):
            speaker_path = os.path.join(condition_path, speaker_folder)
            output_speaker_path = os.path.join(output_condition_path, speaker_folder)

            for video_folder in os.listdir(speaker_path):
                video_path = os.path.join(speaker_path, video_folder)
                csv_file = os.path.join(video_path, "speakers_info.csv")

                if os.path.exists(csv_file):
                    df = pd.read_csv(csv_file)

                    # Strip column names
                    df.columns = df.columns.str.strip()

                    # Check if the required column exists
                    if "years_from_diagnosis" not in df.columns:
                        print(f"Columns in {csv_file}: {df.columns.tolist()}")
                        print(f"Skipping {csv_file}: 'years_from_diagnosis' column missing")
                        continue

                    # Convert to numeric
                    df["years_from_diagnosis"] = pd.to_numeric(df["years_from_diagnosis"], errors="coerce")

                    # Find target speaker with pre-diagnosis recordings
                    df_target = df[(df["status"] == "target") & (df["before_after_diagnosis"] == "before") & (df["years_from_diagnosis"] > 0)]

                    if not df_target.empty:
                        target_speaker = df_target.iloc[0]['speakers']
                        time_before_diagnosis = df_target.iloc[0]['years_from_diagnosis']  # Extract time before diagnosis

                        print(f"Keeping {video_folder} (Speaker: {target_speaker}), Time before diagnosis: {time_before_diagnosis:.2f} years")

                        # Create the output directory
                        output_video_path = os.path.join(output_speaker_path, video_folder)
                        os.makedirs(output_video_path, exist_ok=True)

                        # Copy the target speaker's folder correctly (without deep nesting)
                        target_speaker_path = os.path.join(video_path, target_speaker)
                        if os.path.exists(target_speaker_path):
                            shutil.copytree(target_speaker_path, os.path.join(output_video_path, target_speaker), dirs_exist_ok=True)

                        # Copy required files
                        for file_name in [f"{video_folder}.csv", f"{video_folder}.json", "speakers_info.csv"]:
                            file_path = os.path.join(video_path, file_name)
                            if os.path.exists(file_path):
                                shutil.copy(file_path, output_video_path)

                    else:
                        # Find years from diagnosis that are <= 0 (pre-diagnosis)
                        non_positive_values = df.loc[df["years_from_diagnosis"] == 0, "years_from_diagnosis"].dropna().tolist()

                        # Find years after diagnosis (post-diagnosis)
                        post_diagnosis_values = df.loc[df["years_from_diagnosis"] > 0, "years_from_diagnosis"].dropna().tolist()

                        if post_diagnosis_values:
                            print(f"Skipping {video_folder} (Post-diagnosis)")
                        else:
                            print(f"Skipping {video_folder} (No valid pre-diagnosis target), years from diagnosis: {non_positive_values}")


In [ ]:
filter_target_speakers("ParkCeleb/", "ParkCeleb_filtered/")

Skipping 1G2l17Fp-QI (Post-diagnosis)
Skipping 1ZmRHDKAJec (Post-diagnosis)
Keeping 6a5lxe7LqHk (Speaker: SPEAKER_02), Time before diagnosis: 29.00 years
Skipping 7x9Fs5or4us (Post-diagnosis)
Keeping 9IxTsQhAmNU (Speaker: SPEAKER_00), Time before diagnosis: 36.00 years
Skipping abr6CqbNdM4 (Post-diagnosis)
Keeping DuUUO1Kavwk (Speaker: SPEAKER_03), Time before diagnosis: 2.00 years
Skipping Gi11Fgbjqdc (Post-diagnosis)
Keeping HELG9IIqhBE (Speaker: SPEAKER_00), Time before diagnosis: 36.00 years
Skipping l8xAwdbGpcc (Post-diagnosis)
Skipping lQQpu5coRBA (No valid pre-diagnosis target), years from diagnosis: [0.0]
Skipping MkdLCjAd3Bc (Post-diagnosis)
Skipping nW-4wrt332E (No valid pre-diagnosis target), years from diagnosis: [0.0]
Skipping PqQZki7opNg (Post-diagnosis)
Keeping r1GWW5R1VIU (Speaker: SPEAKER_01), Time before diagnosis: 29.00 years
Keeping r8Kmy0flX_w (Speaker: SPEAKER_00), Time before diagnosis: 4.00 years
Skipping RaWh4aqxRmo (Post-diagnosis)
Skipping RSgGOMpM0uI (Post-d

In [ ]:
def extract_number(speaker_folder):
    """Extracts the numeric part from a speaker folder name, assuming format like 'PD_1' or 'CN_5'."""
    match = re.search(r'\d+', speaker_folder)
    return int(match.group()) if match else None

def filter_cn_speakers(pd_output_path, base_cn_path, output_cn_path):
    """
    Filters CN speakers by matching them to existing PD speakers and applying the same filtering criteria.

    Parameters:
    - pd_output_path: Path where PD speakers are already stored.
    - base_cn_path: Path to the original CN dataset.
    - output_cn_path: Path to save the filtered CN dataset.
    """
    if not os.path.exists(output_cn_path):
        os.makedirs(output_cn_path)

    # Get list of PD speaker numbers
    pd_speaker_numbers = set()
    for speaker_folder in os.listdir(pd_output_path):
        speaker_number = extract_number(speaker_folder)
        if speaker_number is not None:
            pd_speaker_numbers.add(speaker_number)

    print(f"PD speaker numbers found: {sorted(pd_speaker_numbers)}")

    # Process CN speakers that match PD numbers
    for speaker_folder in os.listdir(base_cn_path):
        speaker_number = extract_number(speaker_folder)
        if speaker_number not in pd_speaker_numbers:
            continue  # Skip CN speakers that don't match a PD speaker

        speaker_path = os.path.join(base_cn_path, speaker_folder)
        output_speaker_path = os.path.join(output_cn_path, speaker_folder)

        for video_folder in os.listdir(speaker_path):
            video_path = os.path.join(speaker_path, video_folder)
            output_video_path = os.path.join(output_speaker_path, video_folder)

            csv_file = os.path.join(video_path, "speakers_info.csv")

            if not os.path.exists(csv_file):
                print(f"Skipping {video_folder} (No 'speakers_info.csv' found)")
                continue

            df = pd.read_csv(csv_file)

            # Strip column names
            df.columns = df.columns.str.strip()

            # Check if required column exists
            if "years_from_diagnosis" not in df.columns:
                print(f"Skipping {video_folder}: 'years_from_diagnosis' column missing")
                continue

            # Convert to numeric
            df["years_from_diagnosis"] = pd.to_numeric(df["years_from_diagnosis"], errors="coerce")

            # Find target speaker with pre-diagnosis recordings
            df_target = df[(df["status"] == "target") & (df["before_after_diagnosis"] == "before") & (df["years_from_diagnosis"] > 0)]

            if not df_target.empty:
                target_speaker = df_target.iloc[0]['speakers']
                time_before_diagnosis = df_target.iloc[0]['years_from_diagnosis']  # Extract time before diagnosis

                print(f"Keeping {video_folder} (Speaker: {target_speaker}), Time before diagnosis: {time_before_diagnosis:.2f} years")

                # Create the output directory
                os.makedirs(output_video_path, exist_ok=True)

                # Copy the target speaker's folder correctly (without deep nesting)
                target_speaker_path = os.path.join(video_path, target_speaker)
                if os.path.exists(target_speaker_path):
                    shutil.copytree(target_speaker_path, os.path.join(output_video_path, target_speaker), dirs_exist_ok=True)

                # Copy required files
                for file_name in [f"{video_folder}.csv", f"{video_folder}.json", "speakers_info.csv"]:
                    file_path = os.path.join(video_path, file_name)
                    if os.path.exists(file_path):
                        shutil.copy(file_path, output_video_path)

            else:
                # Find years from diagnosis that are <= 0 (pre-diagnosis)
                non_positive_values = df.loc[df["years_from_diagnosis"] == 0, "years_from_diagnosis"].dropna().tolist()

                # Find years after diagnosis (post-diagnosis)
                post_diagnosis_values = df.loc[df["years_from_diagnosis"] > 0, "years_from_diagnosis"].dropna().tolist()

                if post_diagnosis_values:
                    print(f"Skipping {video_folder} (Post-diagnosis)")
                else:
                    print(f"Skipping {video_folder} (No valid pre-diagnosis target), years from diagnosis: {non_positive_values}")


In [ ]:
pd_output_path = "ParkCeleb_filtered/PD"
base_cn_path = "ParkCeleb/CN"
output_cn_path = "ParkCeleb_filtered/CN"

filter_cn_speakers(pd_output_path, base_cn_path, output_cn_path)


PD speaker numbers found: [1, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
Keeping 4zHrnCPJTFM (Speaker: SPEAKER_01), Time before diagnosis: 34.00 years
Skipping 5iEgR25lS5A (Post-diagnosis)
Keeping eBrxC4jqbbo (Speaker: SPEAKER_01), Time before diagnosis: 34.00 years
Skipping EN8ZdMf5oAQ (No valid pre-diagnosis target), years from diagnosis: [0.0]
Skipping fdLF9NViKl4 (Post-diagnosis)
Skipping FE6QA-n36W4 (Post-diagnosis)
Skipping FZBZ5KmKdX8 (Post-diagnosis)
Keeping HoBQclABghY (Speaker: SPEAKER_00), Time before diagnosis: 4.00 years
Keeping hTOX1l4UEtE (Speaker: SPEAKER_03), Time before diagnosis: 4.00 years
Skipping hznoXSPobHA (No valid pre-diagnosis target), years from diagnosis: [0.0]
Skipping L2tGjae1WD0 (Post-diagnosis)
Skipping metadata.xlsx (No 'speakers_info.csv' found)
Keeping oFUcQ_XgToI (Speaker: SPEAKER_01), Time before diagnosis: 6.00 years
Skipping OS1lvL63mfc (No valid pre-diagnosis target), years 

In [ ]:
ls

ParkCeleb/  ParkCeleb_filtered/  SSL4PR/


In [ ]:
def generate_rsync_file_list(root_dir, output_file):
    """
    Recursively finds all files in `root_dir` and saves their relative paths in `output_file`.

    Parameters:
    - root_dir (str): The base directory where CN and PD files are stored.
    - output_file (str): The file where relative paths will be saved.
    """
    with open(output_file, "w") as f:
        for dirpath, _, filenames in os.walk(root_dir):
            for file in filenames:
                rel_path = os.path.relpath(os.path.join(dirpath, file), root_dir)
                f.write(rel_path + "\n")

    print(f"Rsync file list saved to {output_file}")

generate_rsync_file_list("ParkCeleb_filtered", "parkceleb_filtered_list.txt")


Rsync file list saved to parkceleb_filtered_list.txt


# Preparing for presentation on methodology and background

In [ ]:
def count_wav_files(base_path):
    total_wavs = 0
    concatenated_wavs = 0
    diagnosis_time_counts = defaultdict(int)

    for root, dirs, files in os.walk(base_path):
        # Count .wav files and concatenated ones
        for file in files:
            if file.endswith(".wav"):
                total_wavs += 1
                if file == "recording_concatenated.wav":
                    concatenated_wavs += 1

        # If this folder contains speakers_info.csv, analyze diagnosis time
        if "speakers_info.csv" in files:
            csv_path = os.path.join(root, "speakers_info.csv")
            try:
                df = pd.read_csv(csv_path)
                df.columns = df.columns.str.strip()

                # Convert to numeric just in case
                df["years_from_diagnosis"] = pd.to_numeric(df["years_from_diagnosis"], errors="coerce")

                # Filter valid "before" samples for target speakers
                df_target = df[(df["status"] == "target") &
                               (df["before_after_diagnosis"] == "before") &
                               (df["years_from_diagnosis"] > 0)]

                if not df_target.empty:
                    # Use the first matching one
                    time = df_target.iloc[0]["years_from_diagnosis"]
                    diagnosis_time_counts[time] += 1

            except Exception as e:
                print(f"Error reading {csv_path}: {e}")

    # Print summary
    print(f"Total .wav files: {total_wavs}")
    print(f"'recording_concatenated.wav' files: {concatenated_wavs}")
    print("Samples per time before diagnosis:")
    for k in sorted(diagnosis_time_counts):
        print(f"  {k:.2f} years: {diagnosis_time_counts[k]} sample(s)")

# Usage
count_wav_files("ParkCeleb_filtered")


Total .wav files: 14057
'recording_concatenated.wav' files: 365
Samples per time before diagnosis:
  1.00 years: 45 sample(s)
  2.00 years: 42 sample(s)
  3.00 years: 36 sample(s)
  4.00 years: 39 sample(s)
  5.00 years: 24 sample(s)
  6.00 years: 41 sample(s)
  7.00 years: 21 sample(s)
  8.00 years: 17 sample(s)
  9.00 years: 11 sample(s)
  10.00 years: 23 sample(s)
  11.00 years: 8 sample(s)
  12.00 years: 8 sample(s)
  13.00 years: 8 sample(s)
  14.00 years: 6 sample(s)
  15.00 years: 4 sample(s)
  16.00 years: 4 sample(s)
  17.00 years: 5 sample(s)
  18.00 years: 4 sample(s)
  19.00 years: 2 sample(s)
  20.00 years: 2 sample(s)
  21.00 years: 2 sample(s)
  22.00 years: 3 sample(s)
  23.00 years: 5 sample(s)
  24.00 years: 1 sample(s)
  26.00 years: 2 sample(s)
  27.00 years: 2 sample(s)
  29.00 years: 4 sample(s)
  30.00 years: 2 sample(s)
  31.00 years: 3 sample(s)
  32.00 years: 2 sample(s)
  33.00 years: 1 sample(s)
  34.00 years: 3 sample(s)
  36.00 years: 3 sample(s)
  39.00 y

In [ ]:
def count_wav_files_grouped(base_path):
    total_wavs = 0
    concatenated_wavs = 0
    diagnosis_time_wav_counts = []

    for root, dirs, files in os.walk(base_path):
        # Process speakers_info.csv first
        if "speakers_info.csv" in files:
            csv_path = os.path.join(root, "speakers_info.csv")
            try:
                df = pd.read_csv(csv_path)
                df.columns = df.columns.str.strip()
                df["years_from_diagnosis"] = pd.to_numeric(df["years_from_diagnosis"], errors="coerce")

                df_target = df[(df["status"] == "target") &
                              (df["before_after_diagnosis"] == "before") &
                              (df["years_from_diagnosis"] > 0)]

                if not df_target.empty:
                    years_from_dx = df_target.iloc[0]["years_from_diagnosis"]

                    wav_count = 0
                    # Search the subdirectories for WAVs
                    for sub_root, sub_dirs, sub_files in os.walk(root):
                        wav_files = [f for f in sub_files if f.endswith(".wav")]
                        wav_count += len(wav_files)
                        total_wavs += len(wav_files)
                        concatenated_wavs += sum(1 for f in wav_files if f == "recording_concatenated.wav")

                    # print(f"Found valid record: {years_from_dx} years, {wav_count} wavs in {root}")
                    diagnosis_time_wav_counts.append((years_from_dx, wav_count))

            except Exception as e:
                print(f"Error reading {csv_path}: {e}")

    # Bin the .wav files using diagnosis times and count samples per bin
    if diagnosis_time_wav_counts:
        years_list = [t[0] for t in diagnosis_time_wav_counts]
        bins = range(0, 13, 1)
        labels = [f"{b} years" for b in bins[:-1]]

        # Expand the binned list by repeating the wav_count per bin
        binned_counts = Counter()
        for years, count in diagnosis_time_wav_counts:
            bin_label = pd.cut([years], bins=bins, labels=labels, right=False)[0]
            if pd.notna(bin_label):
                binned_counts[bin_label] += count

        print(f"\nTotal .wav files: {total_wavs}")
        print(f"'recording_concatenated.wav' files: {concatenated_wavs}")
        print("\nSamples grouped by years from diagnosis (5-year bins):")
        for label in labels:
            print(f"  {label}: {binned_counts.get(label, 0)} sample(s)")
    else:
        print("No valid pre-diagnosis target samples found.")

# Example usage
count_wav_files_grouped("ParkCeleb_filtered")



Total .wav files: 10897
'recording_concatenated.wav' files: 282

Samples grouped by years from diagnosis (5-year bins):
  0 years: 0 sample(s)
  1 years: 2095 sample(s)
  2 years: 1696 sample(s)
  3 years: 1026 sample(s)
  4 years: 1186 sample(s)
  5 years: 449 sample(s)
  6 years: 1719 sample(s)
  7 years: 430 sample(s)
  8 years: 342 sample(s)
  9 years: 720 sample(s)
  10 years: 1234 sample(s)
  11 years: 0 sample(s)


In [ ]:
import os
import pandas as pd
import shutil

def clean_dataset_by_diagnosis_time(base_path, max_years=10):
    wav_files_kept = 0
    wav_files_deleted = 0

    # Dictionary to store years_from_diagnosis for each directory
    dir_to_years = {}

    print(f"Scanning for diagnosis information...")

    # First pass: collect diagnosis information
    for root, dirs, files in os.walk(base_path):
        if "speakers_info.csv" in files:
            csv_path = os.path.join(root, "speakers_info.csv")
            try:
                df = pd.read_csv(csv_path)
                df.columns = df.columns.str.strip()
                df["years_from_diagnosis"] = pd.to_numeric(df["years_from_diagnosis"], errors="coerce")

                df_target = df[(df["status"] == "target") &
                              (df["before_after_diagnosis"] == "before") &
                              (df["years_from_diagnosis"] > 0)]

                if not df_target.empty:
                    years_from_dx = df_target.iloc[0]["years_from_diagnosis"]
                    dir_to_years[root] = years_from_dx

            except Exception as e:
                print(f"Error reading {csv_path}: {e}")

    # Second pass: handle WAV files based on diagnosis time
    for root, dirs, files in os.walk(base_path):
        # Find the closest parent directory with diagnosis info
        current_dir = root
        years_from_dx = None

        while current_dir.startswith(base_path):
            if current_dir in dir_to_years:
                years_from_dx = dir_to_years[current_dir]
                break
            parent_dir = os.path.dirname(current_dir)
            if parent_dir == current_dir:  # Reached root
                break
            current_dir = parent_dir

        # Process WAV files in this directory
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)

                if years_from_dx is None or years_from_dx > max_years:
                    try:
                        os.remove(file_path)
                        wav_files_deleted += 1
                        if wav_files_deleted % 1000 == 0:
                            print(f"Deleted {wav_files_deleted} WAV files so far...")
                    except Exception as e:
                        print(f"Error deleting {file_path}: {e}")
                else:
                    wav_files_kept += 1

    # Third pass: delete empty directories
    dirs_deleted = 0
    for root, dirs, files in os.walk(base_path, topdown=False):
        if not os.listdir(root):
            try:
                os.rmdir(root)
                dirs_deleted += 1
            except Exception as e:
                print(f"Error removing directory {root}: {e}")

    print(f"Cleaning complete.")
    print(f"Kept {wav_files_kept} WAV files (≤ {max_years} years before diagnosis)")
    print(f"Deleted {wav_files_deleted} WAV files (> {max_years} years before diagnosis)")
    print(f"Removed {dirs_deleted} empty directories")

# Execute the function
clean_dataset_by_diagnosis_time("ParkCeleb_filtered", max_years=10)

Scanning for diagnosis information...
Deleted 1000 WAV files so far...
Deleted 2000 WAV files so far...
Deleted 3000 WAV files so far...
Cleaning complete.
Kept 10897 WAV files (≤ 10 years before diagnosis)
Deleted 3160 WAV files (> 10 years before diagnosis)
Removed 83 empty directories


# Convert for SSL4PR

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def get_patient_sessions(base_path):
    """
    Collects all .wav file paths grouped by speaker.
    Returns: dict[speaker_id] = list of (label, full_path_to_wav)
    """
    data = {}
    for label in ["PD", "CN"]:
        label_path = os.path.join(base_path, label)
        for speaker in os.listdir(label_path):
            speaker_path = os.path.join(label_path, speaker)
            if not os.path.isdir(speaker_path):
                continue
            for session in os.listdir(speaker_path):
                session_path = os.path.join(speaker_path, session)
                for speaker_folder in os.listdir(session_path):
                    if speaker_folder.startswith("SPEAKER"):
                        speaker_audio_path = os.path.join(session_path, speaker_folder)
                        for recording in os.listdir(speaker_audio_path):
                            if recording.endswith(".wav"):
                                file_path = os.path.join(speaker_audio_path, recording)
                                data.setdefault(speaker, []).append((label, file_path))
    return data

def generate_10_folds(data, output_root="pcgita_splits"):
    speakers = list(data.keys())
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    os.makedirs(output_root, exist_ok=True)

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(speakers), start=1):
        train_speakers = [speakers[i] for i in train_idx]
        test_speakers = [speakers[i] for i in test_idx]

        # Collect paths, labels, and add a placeholder for 'UPDRS-speech'
        train_data = [(label, file_path, 0) for spk in train_speakers for label, file_path in data[spk]]  # 0 as placeholder
        test_data = [(label, file_path, 0) for spk in test_speakers for label, file_path in data[spk]]  # 0 as placeholder

        # Convert to DataFrame to save CSV with columns for label, file path, and UPDRS-speech
        train_df = pd.DataFrame(train_data, columns=["status", "audio_path", "UPDRS-speech"])
        test_df = pd.DataFrame(test_data, columns=["status", "audio_path", "UPDRS-speech"])

        fold_dir = os.path.join(output_root, f"TRAIN_TEST_{fold_idx}")
        os.makedirs(fold_dir, exist_ok=True)

        # Save the data with labels and 'UPDRS-speech'
        train_df.to_csv(os.path.join(fold_dir, "train.csv"), index=False)
        test_df.to_csv(os.path.join(fold_dir, "test.csv"), index=False)

        print(f"✅ Fold {fold_idx} saved to {fold_dir} ({len(train_data)} train, {len(test_data)} test)")

# Run the function to generate the folds with the 'UPDRS-speech' placeholder
generate_10_folds(get_patient_sessions("ParkCeleb_filtered"))


✅ Fold 1 saved to pcgita_splits/TRAIN_TEST_1 (12445 train, 1612 test)
✅ Fold 2 saved to pcgita_splits/TRAIN_TEST_2 (13006 train, 1051 test)
✅ Fold 3 saved to pcgita_splits/TRAIN_TEST_3 (12818 train, 1239 test)
✅ Fold 4 saved to pcgita_splits/TRAIN_TEST_4 (12229 train, 1828 test)
✅ Fold 5 saved to pcgita_splits/TRAIN_TEST_5 (13143 train, 914 test)
✅ Fold 6 saved to pcgita_splits/TRAIN_TEST_6 (13328 train, 729 test)
✅ Fold 7 saved to pcgita_splits/TRAIN_TEST_7 (13045 train, 1012 test)
✅ Fold 8 saved to pcgita_splits/TRAIN_TEST_8 (11706 train, 2351 test)
✅ Fold 9 saved to pcgita_splits/TRAIN_TEST_9 (11168 train, 2889 test)
✅ Fold 10 saved to pcgita_splits/TRAIN_TEST_10 (13625 train, 432 test)


# SSL4PR Replication

In [ ]:
# SSL4PR WavLM Base
! git clone https://huggingface.co/morenolq/SSL4PR-wavlm-base

# SSL4PR HuBERT Base
! git clone https://huggingface.co/morenolq/SSL4PR-hubert-base

Cloning into 'SSL4PR-wavlm-base'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 7), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (32/32), 9.01 KiB | 42.00 KiB/s, done.
Filtering content: 100% (11/11), 3.56 GiB | 42.86 MiB/s, done.
Cloning into 'SSL4PR-hubert-base'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 31 (delta 3), reused 0 (delta 0), pack-reused 21 (from 1)
Unpacking objects: 100% (31/31), 9.74 KiB | 46.00 KiB/s, done.
Filtering content: 100% (11/11), 3.56 GiB | 42.82 MiB/s, done.


In [ ]:
! python SSL4PR/train.py --config SSL4PR/configs/parkceleb_config.yaml

Set all seeds to 42
Class mapping: {'CN': 0, 'PD': 1}
Trying to read test file from: pcgita_splits/TRAIN_TEST_1/test.csv
Does file exist? True
2025-04-07 11:24:21.239743: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 11:24:21.242838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 11:24:21.280952: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 11:24:21.281422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, reb

In [ ]:
import os

def get_label_from_path(file_path):
    # Assuming folders are named 'hc' and 'pd'
    label = os.path.basename(os.path.dirname(file_path))  # Extract the folder name
    if label == 'hc':
        return 0  # Healthy control
    elif label == 'pd':
        return 1  # Parkinson's disease
    else:
        return -1  # Unknown or error

def get_dataloaders(train_path, test_path, class_mapping, config):
    # Read CSV with paths to .wav files
    df_test = pd.read_csv(test_path)

    # Add a new column for labels based on the file paths
    df_test['label'] = df_test['wav_path'].apply(get_label_from_path)

    # Proceed with the rest of your data loading logic
    # For example, convert to a tensor or whatever is needed for your model
    return df_test

def fix_labels(df):
    if 'label' not in df.columns:
        raise KeyError('Label column is missing!')

    # Example: handle missing labels (if any)
    df['label'] = df['label'].fillna('unknown')  # Or any other handling logic
    return df



Checking file path: F:/ASR/Project/pcgita_splits/TRAIN_TEST_1/test.csv
File not found at: F:/ASR/Project/pcgita_splits/TRAIN_TEST_1/test.csv


In [ ]:
!ls


F:  ParkCeleb_filtered	pcgita_splits  split.json  SSL4PR  SSL4PR-hubert-base  SSL4PR-wavlm-base


# Whisper